In [9]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch.utils.data import DataLoader

from Dataset.Economy_Dataset import Economy_Dataset
from Dataset.RNN_Transaction_Dataset import RNN_Transaction_Dataset
from Dataset.ODE_Transaction_Dataset import ODE_Transaction_Dataset

from Model.LSTM import LSTM
from Model.NODE import NODE
from Model.ODERNN import *
from Model.ODEF import *

from utils import preprocess

In [11]:
# 데이터 다운 & 전처리

transaction_df = pd.read_csv('../데이터/transaction/transaction_final.csv')
economy_df = pd.read_excel('../데이터/economy/economy_all.xlsx')

transaction_df, economy_df = preprocess(transaction_df, economy_df, window_size=5)

In [12]:
input_size = 5
hidden_size = 256
output_size = 1

lr = 1e-7
num_epochs = 3

### 부동산 & 경제

In [3]:
transaction_df = pd.read_excel('../데이터/Transaction/transaction_final.xlsx', index_col=0)
economy_df = pd.read_excel('../데이터/Economy/economy_all.xlsx')
economy_df = economy_df['국고채금리']

In [4]:
trainsaction_train_size = int(len(transaction_df)*0.7)
trainsaction_val_size = int(len(transaction_df)*0.3)

transaction_train_dataset = RNN_Transaction_Dataset(transaction_df[:trainsaction_train_size])
transaction_train_loader = DataLoader(transaction_train_dataset, batch_size=2)
transaction_val_dataset = RNN_Transaction_Dataset(transaction_df[trainsaction_train_size:])
transaction_val_loader = DataLoader(transaction_val_dataset, batch_size=2)

economy_train_size = int(len(economy_df)*0.7)
economy_val_size = int(len(economy_df)*0.3)

economy_train_dataset = Economy_Dataset(economy_df[:economy_train_size])
economy_train_loader = DataLoader(economy_train_dataset, batch_size=2)
economy_val_dataset = Economy_Dataset(economy_df[economy_train_size:])
economy_val_loader = DataLoader(economy_val_dataset, batch_size=2)

c:\Users\USER\Desktop\sci\SCI\코드\Dataset\RNN_Transaction_Dataset.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['계약년월'] = pd.to_datetime(data['계약년월'])
c:\Users\USER\Desktop\sci\SCI\코드\Dataset\RNN_Transaction_Dataset.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['계약년월'] = pd.to_datetime(data['계약년월'])
c:\Users\USER\Desktop\sci\SCI\코드\Dataset\Economy_Dataset.py:11: UserWarning: Failed to initialize NumPy: module compiled against API version 0x10 but this version of numpy is 0xe (Triggered 

In [6]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

### 경제 모델

In [16]:
best_val_loss = float('inf') 
for epoch in range(num_epochs + 1):
    model.train()
    for batch_idx, samples in enumerate(economy_train_loader):
        economy_x_train, economy_y_train = samples

        prediction, hidden = model(economy_x_train)
        cost = criterion(prediction, economy_y_train)

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()
    
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for batch_idx, samples in enumerate(economy_val_loader):
            economy_x_val, economy_y_val = samples

            prediction, hidden = model(economy_x_val)
            loss = criterion(prediction, economy_y_val)
            val_loss += loss.item()

    val_loss /= len(economy_val_loader)
    print(f'Epoch {epoch}/{num_epochs}, Training Loss: {cost.item()}, Validation Loss: {val_loss}')
    
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), '../데이터/Checkpoint/best_rnn_economy_model.pth')

Epoch 0/1, Training Loss: 0.002730900188907981, Validation Loss: 0.3143093644015106
Epoch 1/1, Training Loss: 0.0027428490575402975, Validation Loss: 0.31638226120186774


### 부동산 모델

In [7]:
best_val_loss = float('inf') 
for epoch in range(num_epochs + 1):
    model.train()
    for batch_idx, samples in enumerate(transaction_train_loader):
        dong_x_train, dong_y_train = samples

        prediction, hidden = model(dong_x_train)
        cost = criterion(prediction, dong_y_train)

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()
    
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for batch_idx, samples in enumerate(transaction_val_loader):
            x_val, y_val = samples

            prediction, hidden = model(x_val)
            loss = criterion(prediction, y_val)
            val_loss += loss.item()

    val_loss /= len(transaction_val_loader)
    print(f'Epoch {epoch}/{num_epochs}, Training Loss: {cost.item()}, Validation Loss: {val_loss}')
    
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), '../데이터/Checkpoint/best_rnn_transaction_model.pth')

Epoch 0/1, Training Loss: 28219.5234375, Validation Loss: 5213844.961625429
Epoch 1/1, Training Loss: 27884.662109375, Validation Loss: 5179137.586078938


### NODE 부동산 돌리기

In [8]:
# 데이터 다운로드
batch_size = 2
transaction_df = pd.read_excel('../데이터/Transaction/transaction_final.xlsx', index_col=0)

train_dataset = ODE_Transaction_Dataset(transaction_df)
train_loader = DataLoader(train_dataset, batch_size=batch_size)

In [4]:
for x,y,z,w in train_loader:
      print("X 크기 : {}".format(x.shape))
      print("Y 크기 : {}".format(y.shape))
      print("Z 크기 : {}".format(z.shape))
      print("W 크기 : {}".format(w.shape))
      break

X 크기 : torch.Size([2, 5])
Y 크기 : torch.Size([2, 5])
Z 크기 : torch.Size([2, 1])
W 크기 : torch.Size([2, 1])


In [5]:
for x,y,z,w in train_loader:
      print("X 크기 : {}".format(x.dtype))
      print("Y 크기 : {}".format(y.dtype))
      print("Z 크기 : {}".format(z.dtype))
      print("W 크기 : {}".format(w.dtype))
      break

X 크기 : torch.float32
Y 크기 : torch.float32
Z 크기 : torch.float32
W 크기 : torch.float32


In [6]:
# 데이터 & 모델에 device 붙임!!!
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu') # 윈도우 gpu
# device = torch.device('mps:0' if torch.backends.mps.is_available() else 'cpu') # 맥 gpu
print(f'{device} is available')

model = NODE(output_dim=1,hidden_dim=256,latent_dim=64).to(device)

print('작동하는지 실험')
basic_data = torch.rand((5,2,1)).to(device)  # window_size, batch_size, 1
time = torch.FloatTensor([[1,2,3,6,10,12],[1,3,5,8,10,12]]).reshape(6,2,1).to(device) # window_size, batch_size, 1
data = model(basic_data,time)
print(data)
print(data[0].shape)

cpu is available
작동하는지 실험
(tensor([[[0.3200],
         [0.3020]],

        [[0.3165],
         [0.3053]],

        [[0.3074],
         [0.3091]],

        [[0.3381],
         [0.3102]],

        [[0.3129],
         [0.3069]]], grad_fn=<SliceBackward0>), tensor([[[-1.9246,  0.9916,  0.5816,  ..., -0.0953, -0.4212,  0.2555],
         [-0.3911,  0.1918,  0.4691,  ..., -0.1469, -0.4416,  0.2056]],

        [[-2.3361,  0.9280,  0.5030,  ..., -0.3589, -0.1182, -0.2516],
         [-0.4929,  0.1927, -0.0292,  ...,  0.3396, -0.2586, -0.0658]],

        [[-3.3343,  0.8663,  0.3372,  ..., -1.4191,  0.6670, -1.6950],
         [-0.4960,  0.2321, -0.2474,  ...,  0.5994, -0.1838, -0.2525]],

        [[ 0.8072,  1.3306,  1.7080,  ..., -0.8884, -1.7589,  2.7519],
         [ 0.6183,  0.7583,  1.9381,  ..., -1.3055, -0.8485,  0.8296]],

        [[-2.7293,  0.8898,  0.4417,  ..., -0.7159,  0.1923, -0.7784],
         [-0.5078,  0.2105, -0.1448,  ...,  0.4671, -0.2156, -0.1514]],

        [[ 1.2445,  1.3671

In [7]:
window_size = 5
noise_std = 0.02
optim = torch.optim.Adam(model.parameters(), betas=(0.9, 0.999), lr=0.001)

num_epochs=50

In [8]:
best_train_loss = float('inf') 
for epoch in range(num_epochs + 1):
    print('epoch : ',epoch)
    losses = []
    model.train()
    for batch_idx, samples in enumerate(train_loader):
        tran_x, time_x, tran_y, time_y = samples
        
        t = torch.cat((time_x,time_y),dim=1)  
        tran_x = tran_x.transpose(0,1).unsqueeze(2).to(device)
        t = t.transpose(0,1).unsqueeze(2).to(device)
        
        x_p, _, z, z_mean, z_log_var, pred = model(tran_x, t)
        kl_loss = -0.5 * torch.sum(1 + z_log_var - z_mean**2 - torch.exp(z_log_var), -1)
        loss = 0.5 * ((tran_x-x_p)**2).sum(-1).sum(0) / noise_std**2 + kl_loss
        loss = torch.mean(loss)
        loss /= window_size
        optim.zero_grad()   ############### 여기서 second 오류가 나는지 확인!!!!
        loss.backward()
        optim.step()
        losses.append(loss.item())
        
        if loss < best_train_loss:
            best_train_loss = loss
            torch.save(model.state_dict(), "../데이터/checkpoint/best_ODE_transaction_model.pth")
        
        print('loss : {}, best loss : {}'.format(loss, best_train_loss))
    print('-----------------------------------------------------')

epoch :  0
loss : 11219012608.0, best loss : 11219012608.0
loss : 13275686912.0, best loss : 11219012608.0
loss : 13226204160.0, best loss : 11219012608.0
loss : 11485132800.0, best loss : 11219012608.0
loss : 8940823552.0, best loss : 8940823552.0
loss : 7942053888.0, best loss : 7942053888.0
loss : 8093862912.0, best loss : 7942053888.0
loss : 9578081280.0, best loss : 7942053888.0
loss : 11440328704.0, best loss : 7942053888.0
loss : 13386883072.0, best loss : 7942053888.0
loss : 15193017344.0, best loss : 7942053888.0
loss : 1.5491499810317402e+19, best loss : 7942053888.0
loss : nan, best loss : 7942053888.0
loss : nan, best loss : 7942053888.0
loss : nan, best loss : 7942053888.0
loss : nan, best loss : 7942053888.0
loss : nan, best loss : 7942053888.0
loss : nan, best loss : 7942053888.0
loss : nan, best loss : 7942053888.0
loss : nan, best loss : 7942053888.0
loss : nan, best loss : 7942053888.0
loss : nan, best loss : 7942053888.0
loss : nan, best loss : 7942053888.0
loss : na

KeyboardInterrupt: 

### ODERNN 부동산 돌리기

In [13]:
batch_size = 1

train_dataset = ODE_Transaction_Dataset(transaction_df)
train_loader = DataLoader(train_dataset, batch_size=batch_size)

In [33]:
# 인덱스 번호가 안나와야 됨!!!

for i, (x,y,z,w) in enumerate(train_loader):
    time_f = torch.cat([y,w],axis=1)
    if (time_f[0][1:]>time_f[0][:-1]).all() == False:
        print(i)
        break
    
print(y)
print(w)

1264


In [5]:
for x,y,z,w in train_loader:
      print("X 크기 : {}".format(x.shape))
      print("Y 크기 : {}".format(y.shape))
      print("Z 크기 : {}".format(z.shape))
      print("W 크기 : {}".format(w.shape))
      break

X 크기 : torch.Size([1, 5])
Y 크기 : torch.Size([1, 5])
Z 크기 : torch.Size([1, 1])
W 크기 : torch.Size([1, 1])


In [24]:
# 데이터 & 모델에 device 붙임!!!
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f'{device} is available')

model = ODE_RNN(256,64,1,200,200,device).to(device)

print('작동하는지 실험')
t = torch.FloatTensor([[1.,2.,3.,5.,9.,9.]]).to(device)
data = torch.FloatTensor([[123,126,266,279,300]]).to(device)

out = model(data,t)
print(out[0])

cpu is available
작동하는지 실험


AssertionError: t must be strictly increasing or decreasing

#### 학습하기

In [7]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [10]:
best_train_loss = float('inf') 
for epoch in range(num_epochs + 1):
    print('epoch : ',epoch)
    losses = []
    model.train()
    for batch_idx, samples in enumerate(train_loader):
        tran_x, time_x, tran_y, time_y = samples
        tran_x, time_x, tran_y, time_y = tran_x.to('cuda:0'), time_x.to('cuda:0'), tran_y.to('cuda:0'), time_y.to('cuda:0')
        time_f = torch.cat([time_x,time_y],axis=1)
        prediction, hidden = model(tran_x,time_f)
        cost = criterion(prediction[:,:5], tran_x)

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()
    
    print(f'Epoch {epoch}/{num_epochs}, Training Loss: {cost.item()}')
    
    # model.eval()
    # val_loss = 0.0
    # with torch.no_grad():
    #     for batch_idx, samples in enumerate(transaction_val_loader):
    #         x_val, y_val = samples

    #         prediction, hidden = model(x_val)
    #         loss = criterion(prediction, y_val)
    #         val_loss += loss.item()

    # val_loss /= len(transaction_val_loader)
    # print(f'Epoch {epoch}/{num_epochs}, Training Loss: {cost.item()}, Validation Loss: {val_loss}')
    
    # if val_loss < best_val_loss:
    #     best_val_loss = val_loss
    #     torch.save(model.state_dict(), '../데이터/Checkpoint/best_rnn_transaction_model.pth')

epoch :  0


AssertionError: t must be strictly increasing or decreasing

### ODERNN 실험